In [1]:
from huggingface_hub import notebook_login, interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to C:\Users\katko\.cache\huggingface\token
Login successful


In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [3]:
task = "srl" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Preprocessing

In [5]:
from preprocessing import extract_data_from_conll_extended
extract_data_from_conll_extended("../data/en_ewt-up-test.conllu", "dummy_train.json")

In [6]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json', data_files={'train': 'dummy_train.json', 'test': 'dummy_test.json'})
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['word', 'predicate_position', 'predicate', 'arguments'],
        num_rows: 4799
    })
    test: Dataset({
        features: ['word', 'predicate_position', 'predicate', 'arguments'],
        num_rows: 12
    })
})

In [7]:
datasets["train"][0]

{'word': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?'],
 'predicate_position': 4,
 'predicate': 'morph.01',
 'arguments': ['_', '_', 'ARG1', 'V', '_', 'ARG2', '_']}

In [8]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [9]:
show_random_elements(datasets["train"])

,word,predicate_position,predicate,arguments
0,"[Think, that, was, bad, -, by, December, of, '74, the, Dow, had, sunk, to, a, bear, market, low, of, 577, .]",12,have.01,"[_, _, _, _, _, _, _, _, _, _, _, V, _, _, _, _, _, _, _, _, _]"
1,"[If, you, do, not, wish, to, receive, such, e-mails, in, the, future, or, want, to, know, more, about, the, BBC, 's, Email, a, Friend, service, ,, please, read, our, frequently, asked, questions, .]",7,receive.01,"[_, ARG0, _, _, _, _, V, _, ARG1, _, _, ARGM-TMP, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]"
2,"[In, addition, ,, is, there, someone, who, could, consider, the, correlation, between, US, CPI, and, the, $, /, euro, exchange, rate, .]",11,correlate.01,"[_, _, _, _, _, _, _, _, _, _, V, _, _, ARG1, _, _, _, _, _, _, _, _]"
3,"[If, you, can, afford, to, go, before, ,, then, by, all, means, ,, GO, .]",6,go.02,"[_, ARG0, _, _, _, V, ARGM-TMP, _, _, _, _, _, _, _, _]"
4,"[He, makes, some, good, observations, on, a, few, of, the, pic's, .]",2,make.LV,"[_, V, _, _, ARGM-PRR, _, _, _, _, _, _, _]"
5,"[can, ever, &, never, forget, the, training, undergone, here, which, made, my, life, step, onto, the, successful, job, without, any, hurdles, .]",11,make.02,"[_, _, _, _, _, _, ARG0, _, _, R-ARG0, V, _, ARG1, ARG1, _, _, _, _, _, _, _, _]"
6,"[The, sample, includes, our, "", caveat, "", ,, so, it, should, be, fine, .]",13,fine.04,"[_, _, _, _, _, _, _, _, ARGM-ADV, ARG1, ARGM-MOD, _, V, _]"
7,"[Then, it, 's, time, for, my, nap, .]",7,nap.01,"[_, _, _, _, _, ARG0, V, _]"
8,"[WE, HAVE, A, DATE, FOR, THE, RELEASE, OF, RAGNAROK, ONLINE, 2, (, beta, anyway, ), September, 16, -, 18, ,, this, was, announced, by, Gravity, CEO, Kim, Jung, -, Ryool, on, either, 16th, or, 17th, of, july, and, as, i, do, n't, want, to, take, someone, 's, credit's, i, got, it, here, ^^, GameSpot]",50,get.01,"[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, ARGM-CAU, _, _, _, _, _, ARG0, V, ARG1, ARGM-LOC, _, _]"
9,"[Go, in, and, you, will, not, think, you, are, in, Chesapeake, .]",7,think.01,"[_, _, _, ARG0, ARGM-MOD, ARGM-NEG, V, _, _, _, ARG1, _]"


In [10]:
datasets["train"][0]

{'word': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?'],
 'predicate_position': 4,
 'predicate': 'morph.01',
 'arguments': ['_', '_', 'ARG1', 'V', '_', 'ARG2', '_']}

### Tokenization

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [58]:
def tokenize(instance):
    def align(tokenized_input, labels):
        word_ids = tokenized_input.word_ids()
        aligned_labels = ['_'] * len(word_ids) 
        label_index = 0 
        for i, word_id in enumerate(word_ids):
            try:
                if word_id is None:
                    aligned_labels[i] = -100
                    continue 
                original_label = labels[word_id]
                if original_label == '_':
                    continue 
                if i == 0 or word_id != word_ids[i-1]:
                    prefix = ''
                else:
                    prefix = ''
                aligned_labels[i] = f'{prefix}{original_label}'
            except:
                pass
    
        return aligned_labels
    sentence = instance["word"]
    labels = instance["arguments"]
    tokenized_input = tokenizer(sentence, is_split_into_words=True)
    predicate = instance['predicate']
    predicate = tokenizer(predicate)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    aligned_labels = align(tokenized_input, labels)
    tokenized_input['input_ids'].extend(predicate['input_ids'][1:]), tokenized_input['attention_mask'].extend(predicate['attention_mask'][1:]), aligned_labels.extend(['[PAD]' for i in range(len(predicate['input_ids'][1:]))])

    
    return tokenized_input, tokens, aligned_labels

In [59]:
print(len(datasets['train']))
training_set = [{'x': tokenize(instance)[0], 'base_words':tokenize(instance)[1],'y': tokenize(instance)[2]} for instance in datasets['train']]


4799


In [60]:
sample = training_set[8]
print(len(training_set))
print(len(sample['x']['input_ids']), len(sample['y']))

4799
48 48


In [53]:
sample

{'x': {'input_ids': [101, 2054, 2065, 8224, 22822, 8458, 2098, 2046, 8224, 2891, 1029, 102, 22822, 8458, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 'base_words': ['[CLS]',
  'what',
  'if',
  'google',
  'mor',
  '##ph',
  '##ed',
  'into',
  'google',
  '##os',
  '?',
  '[SEP]'],
 'y': [-100, '_', '_', 'ARG1', 'V', 'V', 'V', '_', 'ARG2', 'ARG2', '_', -100]}

In [ ]:
datasets['train'][2]

In [ ]:
print(tokenized_input.word_ids())


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
label_list = set(label for instance in datasets["train"] for label in instance['arguments'])

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")
metric = load_metric("seqeval")


In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])